In [1]:
import numpy as np

In [2]:
class Agent:
    def __init__(self, id, neighbors, domains, constraints):
        self.id = id
        self.neighbors = neighbors
        self.domain = domains[id]
        self.cost = {}
        for val in self.domain:
            self.cost[val] = np.zeros(len(neighbors) + 1)
        for i, neighbor in enumerate(neighbors):
            for val_i in self.domain:
                for val_j in domains[neighbor]:
                    self.cost[val_i][i] += constraints[self.id, neighbor](val_i, val_j)
            self.cost[val_i][-1] = self.cost[val_i][i]

    def get_best_value(self, values, projection_matrix):
        best_val = None
        best_cost = float('inf')
        for val in self.domain:
            cost = self.cost[val].dot(projection_matrix[self.id, values])
            if cost < best_cost:
                best_cost = cost
                best_val = val
        return best_val

In [3]:
def DPOP(domains, constraints):
    n_agents = len(domains)
    agents = [Agent(i, [j for j in range(n_agents) if j != i], domains, constraints) for i in range(n_agents)]
    projection_matrix = np.zeros((n_agents, len(domains[0]), n_agents + 1))
    for i in range(n_agents):
        for j in range(n_agents):
            projection_matrix[i, :, j] = np.sum(np.array([agents[k].cost[val] for k, val in enumerate(domains[i])]), axis=0)
        projection_matrix[i, :, -1] = projection_matrix[i, :, i]
    values = np.zeros(n_agents, dtype=int)
    for _ in range(n_agents):
        for i in range(n_agents):
            values[i] = agents[i].get_best_value(values, projection_matrix)
        if np.all([agents[i].cost[values[i]].dot(projection_matrix[i, values[i], :-1]) == agents[i].cost[values[i]].dot(projection_matrix[i, values[i], -1]) for i in range(n_agents)]):
            return values
        for i in range(n_agents):
            projection_matrix[i, values[i], :] = 0
            for j in range(n_agents):
                if j != i:
                    projection_matrix[i, values[i], j] = np.sum(np.array([agents[k].cost[val][j] for k, val in enumerate(domains[i])]), axis=0)
            projection_matrix[i, values[i], -1] = projection_matrix[i, values[i], i]
    return values